#Fast Food SVM
Application of the Fast Food kernel expansion algorithm to SVMs.

In [3]:
from __future__ import division, print_function
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pylab as pl
import theano
import theano.tensor as T
import numpy as np
import pandas as pd
import sklearn.datasets
from sklearn import svm
from scipy.linalg import hadamard
from functools import partial

VERBOSE = True  # increases verbosity of some outputs

# random number generator
rng = np.random.RandomState()

# load data
iris = sklearn.datasets.load_iris()
X = iris.data
y = iris.target

# pad with zeros so d is nearest power of 2
N = X.shape[0]
l = X.shape[1]
print('Original shape of X: n =',N,', d =',l)
d = int(2 ** np.ceil(np.log2(l)))
if d != l:  # pad only if needed
    print('Padding input from d =',l,'to d =',d)
    X = np.pad(X,((0,d-l),(0,0)),mode='constant',constant_values=0)

# convert to shared tensors
X = theano.shared(X,name='X',borrow=True)
y = theano.shared(y,name='y',borrow=True)

Original shape of X: n = 150 , d = 4


In [4]:
def generate_ff_model(n,d,sigma):
    def generate_layer():
        # B, binary scaling matrix
        B = np.diag(rng.choice([-1,1],d))  # B_ii from {-1,1} iid

        # G, gaussian random matrix
        G = np.diag(rng.normal(size=d)) # g_ii from N(0,1) iid

        # S, Scaling matrix
        # large values in S_ii correspond to high complexity basis functions
        # whereas small S_ii relate to simple functions with low total variation
        G_frob = 1.0/np.sqrt(np.linalg.norm(G,ord='fro'))
        exp_fact = 1  # -d/2.0  # exponent factor
        area = ((1.0/np.sqrt(d*np.pi)) * (2.0*np.pi*np.exp(exp_fact) / d)) ** (d/2)
        s_i = (2.0*np.pi) ** (-d/2.0) * (1.0/area)
        S_vals = np.empty(d)
        S_vals.fill(s_i*G_frob)
        S = np.diag(S_vals)

        # hadamard matrix, H
        H = hadamard(d,dtype=np.int)

        V = (1.0/(sigma*np.sqrt(d))) * reduce(np.dot,[S,H,G,PI,B])
        return V
    # generate FF transforms
    if N > d:
        # stack enough fastfood transforms for all the samples
        nstack = int(N/d)
        print('Generating',nstack,'stacked FF transforms.')
        ff_stack = np.concatenate([generate_ff_transform('V_'+str(i)) for i in xrange(nstack)])
    else:
        print('Generating a single FF transform')
        ff_stack = generate_ff_transform('V_0')
    if n > d:
        # stack enough V_i to have enough dimensions
        nstack = int(n/d)
        V = np.concat([generate_layer() for _ in xrange(nstack)],axis=1)

def fastfood_rbf_kernel(V,x,y):
    phi = lambda x: (1/np.sqrt(n)) * np.exp(1j * np.dot(V,x))
    k = lambda x,y: np.dot(phi(x),phi(y))
    return k(x,y)

V = generate_ff_model(n,d,sigma)
classifier = svm.SVC(kernel=partial(fastfood_rbf_kernel,V))
classifier.fit(X, Y)

Generating 37 stacked FF transforms.
Generating fast food transform "V_0"
B = [[-1  0  0  0]
 [ 0 -1  0  0]
 [ 0  0 -1  0]
 [ 0  0  0  1]]
PI = [[ 0.  1.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]]
G = [[-0.60375008  0.          0.          0.        ]
 [ 0.         -0.52472381  0.          0.        ]
 [ 0.          0.          0.56498034  0.        ]
 [ 0.          0.          0.         -0.217399  ]]
S = [[ 0.01743163  0.          0.          0.        ]
 [ 0.          0.01743163  0.          0.        ]
 [ 0.          0.          0.01743163  0.        ]
 [ 0.          0.          0.          0.01743163]]
H = [[ 1  1  1  1]
 [ 1 -1  1 -1]
 [ 1  1 -1 -1]
 [ 1 -1 -1  1]]
Generating fast food transform "V_1"
B = [[ 1  0  0  0]
 [ 0  1  0  0]
 [ 0  0  1  0]
 [ 0  0  0 -1]]
PI = [[ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]
G = [[-0.84450629  0.          0.          0.        ]
 [ 0.          1.10574038  0.          0.        ]
 [ 0. 

In [5]:
from mpl_toolkids.mplot3d import Axes3D
fig = plt.figure()
ax = Axes3D(fig)

y = np.arange(100)
y_hat = np.dot(rng.normal(size=100),y)
z = 1.0/np.exp(-np.dot(y,y_hat))

ax.plot_surface(y,y_hat,z)

ImportError: No module named mpl_toolkids.mplot3d